# 1. LET'S FIND THE DISTRIBUTION OF THE ITERATIONS REQUIRED TO FOOL THE NEURAL NETWORK. IT SHOULD IDEALLY FOLLOW A POWER LAW.

In [ ]:
import os
import numpy
import matplotlib.pyplot as plt
import re
import numpy as np
import Image
import pickle
import pretrainedmodels.utils as utils
from skimage.draw import line_aa
import pretrainedmodels
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import requests

LABELS_URL = 'https://s3.amazonaws.com/mlpipes/pytorch-quick-start/labels.json'

labels = {int(key):value for (key, value)
          in requests.get(LABELS_URL).json().items()}

load_img = utils.LoadImage()

resnet_path = 'log_resnet50/'

resnet_log_files = (os.listdir(resnet_path))

smax = nn.Softmax(dim=1)

def prep_image_for_inference(img_path):
    input_img = load_img(img_path)
    input_tensor = tf_img(input_img)
    input_tensor = input_tensor.unsqueeze(0)
    input_to_model = torch.autograd.Variable(input_tensor,
            requires_grad=False)
#     input_to_model -= torch.min(input_to_model)
#     input_to_model = input_to_model / torch.max(input_to_model)
    return input_to_model


# Get the base image, pickle, original class, target class, x,y co-ordinates for a given file path
def get_scratch_details(filepath, model):
    print (filepath)
    # Load the base image name
    image_names = filepath.split('_target')
    image_path = image_names[0]
    
    # Extract the pickle name
    if (model=='resnet50'):
        pickle_path  = 'pickled_resnet50/' + filepath
    elif (model=='inceptionv3'):
        pickle_path = 'pickled_image_parts_abs/' + filepath
    elif (model=='vgg19'):
        pickle_path  = 'pickled_vgg19/' + filepath
    
    # Extract the base image as a numpy array
    imagenetpath = '/home/malhar/Desktop/data/imagenet_validation_images/'
    img_loaded = (Image.open(imagenetpath + image_path))
    if (model=='inceptionv3'):
        img_resized = img_loaded.resize((299, 299))
    else:
        img_resized = img_loaded.resize((224, 224))
        
    img = np.array(img_resized)/255.
    
    # Extract the numbers from the string
    numbers = re.findall(r'\d+', filepath)
    print (numbers)
    target_class = numbers[2]
    original_class = numbers[3]
    x0 = numbers[5]
    y0 = numbers[7]
    x1 = numbers[9]
    y1 = numbers[11]
    
    # Load the pickle
    pickled_scratch = pickle.load(open(pickle_path, "rb" ))
    
    return img, pickled_scratch, target_class, original_class, x0, y0, x1, y1, imagenetpath + image_path

# Get the distribution of iterations required to fool a particular model
def get_iterations_required(model_type, population):
    path = []
    if (model_type == 'resnet50'):
        path = 'log_resnet50/'
    elif (model_type == 'inceptionv3'):
        path = 'log_abs/'
    elif (model_type == 'vgg19'):
        path = 'log_vgg19/'
    log_files = (os.listdir(path))
    
    iterations_required = []

    for i in range(len(log_files)):
        filename = path + log_files[i]
        with open(filename) as f:
            data = f.readlines()
            for line in data:
                if ('Reached' in line):
                    iters = [int(s) for s in line.split() if s.isdigit()]
                    iterations_required.append(iters[0])
    
    print ('Got iterations!')
    return np.array(iterations_required)*population


def get_original_and_scratched(model_name):
    
    if (model_name == 'resnet50'):
        filepath = 'pickled_resnet50/'
    elif (model_name == 'inceptionv3'):
        filepath = 'pickled_image_parts_abs/'
    elif (model_name == 'vgg19'):
        filepath = 'pickled_vgg19/'
        
    files = os.listdir(filepath)
    
    print ('Number of files: ', len(files))

    # Obtain the model
    model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    model.eval()
    tf_img = utils.TransformImage(model)
    

    # Get the image, the pickle, original class, target class, x0,y0, x1, y1 for a given file path

    file_indexer = np.random.randint(0, len(files))

    print (file_indexer)

    bundled = get_scratch_details(files[file_indexer], model_name)

    img = bundled[0]
    scratch = bundled[1]
    target_class = int(bundled[2])
    original_class = int(bundled[3])
    x0 = int(bundled[4])
    y0 = int(bundled[5])
    x1 = int(bundled[6])
    y1 = int(bundled[7])
    base_path = bundled[-1]
    scratch_length = len(scratch)

    # Draw the line on the scratch
    rr, cc, val = line_aa(x0, y0, x1, y1)
    img_scratched = np.copy(img)
    print (img_scratched.shape)
    img_scratched[rr, cc, 0] = scratch[0:int(scratch_length/3)]
    img_scratched[rr, cc, 1] = scratch[int(scratch_length/3):int(2 * scratch_length/3)]
    img_scratched[rr, cc, 2] = scratch[int(2 * scratch_length/3):int(scratch_length)]

    # Let's do inference now bb
    img_for_inference = prep_image_for_inference(base_path)
    img_scratched_for_inference = prep_image_for_inference(base_path)
    scratch_torch = torch.from_numpy(scratch)
    scratch_torch = scratch_torch.type(torch.float)


    img_scratched_for_inference[:,0,rr,cc] = scratch_torch[0:int(scratch_length/3)]
    img_scratched_for_inference[:,1,rr,cc] = scratch_torch[int(scratch_length/3):int(2 * scratch_length/3)]
    img_scratched_for_inference[:,2,rr,cc] = scratch_torch[int(2 * scratch_length/3):int(scratch_length)]

    original_predictions = smax(model(img_for_inference))
    scratch_predictions = smax(model(img_scratched_for_inference))
    top_original_prediction = int(torch.argmax(original_predictions).cpu().numpy())
    top_scratch_prediction = int(torch.argmax(scratch_predictions).cpu().numpy())

    print ('Model: ', model_name)
    print ('Top original class: ', top_original_prediction, 'Original class: ', original_class)
    print ('Top original label: ', labels[top_original_prediction], 'Original class: ', labels[original_class])
    print ('Original prediction confidence: ', torch.max(original_predictions).detach().numpy())
    print ('Top scratch class: ', top_scratch_prediction, 'Target class: ', target_class)
    print ('Top scratch label: ', labels[top_scratch_prediction], 'Original class: ', labels[target_class])
    print ('Scratch prediction confidence: ', torch.max(scratch_predictions).detach().numpy())
    print ('\n')

#     plt.figure()
#     plt.subplot(121)
#     plt.imshow(img[:])
#     plt.title(labels[top_original_prediction]+': ' + str(torch.max(original_predictions).detach().numpy()))
#     plt.axis('off')
#     plt.subplot(122)
#     plt.imshow(img_scratched[:])
#     plt.title(labels[target_class]+': ' +  str(torch.max(scratch_predictions).detach().numpy()))
#     plt.axis('off')


    
    return img, labels[top_original_prediction], (torch.max(original_predictions).detach().numpy()), img_scratched, labels[top_scratch_prediction], (torch.max(scratch_predictions).detach().numpy())





# 1. LET'S FIND THE DISTRIBUTION OF THE ITERATIONS REQUIRED TO FOOL THE NEURAL NETWORK. IT SHOULD IDEALLY FOLLOW A POWER LAW.

In [ ]:
population=80
inception_iters = get_iterations_required('inceptionv3', population)
resnet_iters = get_iterations_required('resnet50', population)
vgg_iters = get_iterations_required('vgg19', population)

In [ ]:
# LET'S PLOT ALL OF IT OUT
BINS = 200
RANGE_LIM = 1000000
FIG_size=(6, 5)
fsize = 15
    


fig = plt.figure(figsize=FIG_size)
ax = fig.add_subplot(111)
plt.hist(inception_iters, bins=BINS, color='r', label='Inception-v3')
plt.xlabel('Total queries required', size=fsize)
plt.xticks(rotation=15)
plt.ylabel('Count', size=fsize)
ax.tick_params(axis='both', which='major', labelsize=fsize)
plt.legend(fontsize=fsize)
fig.tight_layout()
plt.savefig('images/inception_queries.pdf', format='pdf')






fig = plt.figure(figsize=FIG_size)
ax = fig.add_subplot(111)
plt.hist(inception_iters/population, bins=BINS, color='r', label='Inception-v3')
plt.xlabel('Generations required', size=fsize)
plt.xticks(rotation=15)
plt.ylabel('Count', size=fsize)
ax.tick_params(axis='both', which='major', labelsize=fsize)
plt.legend(fontsize=fsize)
fig.tight_layout()
plt.savefig('images/inception_generations.pdf', format='pdf')





fig = plt.figure(figsize=FIG_size)
ax = fig.add_subplot(111)
plt.hist(resnet_iters, bins=BINS, color='b', label='ResNet-50')
plt.xlabel('Total queries required', size=fsize)
plt.xticks(rotation=15)
plt.ylabel('Count', size=fsize)
ax.tick_params(axis='both', which='major', labelsize=fsize)
plt.legend(fontsize=fsize)
fig.tight_layout()
plt.savefig('images/resnet50_queries.pdf', format='pdf')





fig = plt.figure(figsize=FIG_size)
ax = fig.add_subplot(111)
plt.hist(resnet_iters/population, bins=BINS, color='b', label='ResNet-50')
plt.xlabel('Generations required', size=fsize)
plt.xticks(rotation=15)
plt.ylabel('Count', size=fsize)
ax.tick_params(axis='both', which='major', labelsize=fsize)
plt.legend(fontsize=fsize)
fig.tight_layout()
plt.savefig('images/resnet50_generations.pdf', format='pdf')




fig = plt.figure(figsize=FIG_size)
ax = fig.add_subplot(111)
plt.hist(vgg_iters, bins=BINS, color='c', label='VGG-19')
plt.xlabel('Total queries required',size=fsize)
plt.xticks(rotation=15)
ax.tick_params(axis='both', which='major', labelsize=fsize)
plt.ylabel('Count', size=fsize)
plt.legend(fontsize=fsize)
fig.tight_layout()
plt.savefig('images/vgg19_queries.pdf', format='pdf')




fig = plt.figure(figsize=FIG_size)
ax = fig.add_subplot(111)
plt.hist(vgg_iters/population, bins=BINS, color='c', label='VGG-19')
plt.xlabel('Generations required', size=fsize)
plt.xticks(rotation=15)
plt.ylabel('Count', size=fsize)
ax.tick_params(axis='both', which='major', labelsize=fsize)
plt.legend(fontsize=fsize)
fig.tight_layout()
plt.savefig('images/vgg19_generations.pdf', format='pdf')




print '\nInception mean iters: %d'%np.mean(inception_iters)
print 'Inception std iters: %d'%np.std(inception_iters)
print 'Inception median iters: %d'%np.median(inception_iters)

print '\nResNet-50 mean iters: %d'%np.mean(resnet_iters)
print 'ResNet-50 std iters: %d'%np.std(resnet_iters)
print 'ResNet-50 median iters: %d'%np.median(resnet_iters)

print '\nVGG-19 mean iters: %d'%np.mean(vgg_iters)
print 'VGG-19 std iters: %d'%np.std(vgg_iters)
print 'VGG-19 median iters: %d'%np.median(vgg_iters)

print inception_iters

# 2: LET'S NOW GET THE PICKLED REGION FROM THE PICKLE FILE. LET'S THEN PLOT IT OUT. 

# IT WORKS! ONLY FOR VGG-19 AND RESNET-50 THOUGH.

In [ ]:
MODEL_NAME = 'resnet50'

if (MODEL_NAME == 'vgg19'):
    filepath = 'pickled_vgg19/'
elif (MODEL_NAME == 'resnet50'):
    filepath = 'pickled_resnet50/'

files = os.listdir(filepath)

# Obtain the model
model = pretrainedmodels.__dict__[MODEL_NAME](num_classes=1000, pretrained='imagenet')
model.eval()
tf_img = utils.TransformImage(model)

print ('FILES: ', len(files))

# Get the image, the pickle, original class, target class, x0,y0, x1, y1 for a given file path
RANDOS=1
for i in range(RANDOS):
    
    random_class = np.random.randint(0, len(files))
    
    print (random_class)

    bundled = get_scratch_details(files[random_class], MODEL_NAME)

    img = bundled[0]
    scratch = bundled[1]
    
    scratch = (scratch[:] - np.min(scratch)) / (np.max(scratch) - np.min(scratch))
    
    target_class = int(bundled[2])
    original_class = int(bundled[3])
    x0 = int(bundled[4])
    y0 = int(bundled[5])
    x1 = int(bundled[6])
    y1 = int(bundled[7])
    base_path = bundled[-1]
    scratch_length = len(scratch)

    # Draw the line on the scratch
    rr, cc, val = line_aa(x0, y0, x1, y1)
    img_scratched = np.copy(img)
    print (img_scratched.shape)
    if (len(img_scratched.shape) != 3):
        continue
    img_scratched[rr, cc, 0] = scratch[0:int(scratch_length/3)]
    img_scratched[rr, cc, 1] = scratch[int(scratch_length/3):int(2 * scratch_length/3)]
    img_scratched[rr, cc, 2] = scratch[int(2 * scratch_length/3):int(scratch_length)]

    # Let's do inference now bb
    img_for_inference = prep_image_for_inference(base_path)
    img_scratched_for_inference = prep_image_for_inference(base_path)
    scratch_torch = torch.from_numpy(scratch)
    scratch_torch = scratch_torch.type(torch.float)
    
    img_scratched_for_inference[:,0,rr,cc] = scratch_torch[0:int(scratch_length/3)]
    img_scratched_for_inference[:,1,rr,cc] = scratch_torch[int(scratch_length/3):int(2 * scratch_length/3)]
    img_scratched_for_inference[:,2,rr,cc] = scratch_torch[int(2 * scratch_length/3):int(scratch_length)]
    
    original_predictions = smax(model(img_for_inference))
    scratch_predictions = smax(model(img_scratched_for_inference))
    top_original_prediction = int(torch.argmax(original_predictions).cpu().numpy())
    top_scratch_prediction = int(torch.argmax(scratch_predictions).cpu().numpy())
    
    print ('Top original label: ', labels[top_original_prediction], 'Original class: ', labels[original_class])
    print ('Original prediction confidence: ', torch.max(original_predictions).detach().numpy())
    print ('Scratch prediction top_scratch_prediction: ', torch.max(scratch_predictions).detach().numpy())
    print ('Top scratch class: ', top_scratch_prediction, 'Target class: ', target_class)
    print ('Top original class: ', top_original_prediction, 'Original class: ', original_class)
    print ('\n')
    
#     # Plot the combined stuff here
    fig = plt.figure(figsize=(10,10))
#     # Plot benign here
    plt.subplot(121)
    plt.imshow(img[:])
    benign_title =  'Original Image \n'
    benign_title += 'Top prediction: ' + '\n' + labels[top_original_prediction] + '\n'
#     benign_title += 'Probability {:,.2%}'.format(torch.max(original_predictions).detach().numpy())
    plt.title(benign_title, size=15)
    plt.axis('off')
    
#     # Plot scratched here
    plt.subplot(122)
    plt.imshow(img_scratched[:])    
    scratched_title =  'Scratched Image \n'
    scratched_title += 'Top prediction: ' + '\n' + labels[target_class] + '\n'
#     scratched_title += 'Probability {:,.2%}'.format(torch.max(scratch_predictions).detach().numpy())
    plt.title(scratched_title, size=15)
    plt.axis('off')
    
#     plt.imsave('images/IMAGE_' + MODEL_NAME + str(random_class), img_scratched[:])
#     fig.savefig('images/COMBINED_' + MODEL_NAME + str(random_class) + '.pdf', bbox_inches='tight')
    

# LET'S DO IT FOR INCEPTION-V3 NOW

In [ ]:
scratch_torch

In [ ]:
img, top_original_prediction, original_predictions_prob, img_scratched, top_scratched_class, scratched_class_prob = get_original_and_scratched('vgg19')

benign_title =  'Original Image \n'
benign_title += 'Top predicted class: ' + top_original_prediction + '\n'
benign_title += 'Probability {:,.2%}'.format(original_predictions_prob)


scratched_title =  'Scratched Image \n'
scratched_title += 'Top predicted class: ' + top_scratched_class + '\n'
scratched_title += 'Probability {:,.2%}'.format(scratched_class_prob)

plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(img1[:])
plt.title(benign_title, size=15)
plt.axis('off')
plt.subplot(122)
plt.imshow(img_scratched1[:])
plt.title(scratched_title, size=15)
plt.axis('off')

# 3. LET'S NOW GET THE DISTRIBUTION OF THE LINES WHERE THE SCRATCHES OCCURRED. THEY SEEM TO BE UNIFORMLY DISTRIBUTED.

In [ ]:
model_name = 'resnet50'
img = []

if (model_name == 'resnet50'):
    filepath = 'pickled_resnet50/'
    img = np.zeros((224,224))
elif (model_name == 'inceptionv3'):
    filepath = 'pickled_image_parts_abs/'
    img = np.zeros((299,299))
elif (model_name == 'vgg19'):
    filepath = 'pickled_vgg19/'
    img = np.zeros((224,224))

files = os.listdir(filepath)

for i in range(len(files)/2):
    bundled = get_scratch_details(files[i], model_name)

#     img = bundled[0]
    scratch = bundled[1]
    target_class = int(bundled[2])
    original_class = int(bundled[3])
    x0 = int(bundled[4])
    y0 = int(bundled[5])
    x1 = int(bundled[6])
    y1 = int(bundled[7])
    base_path = bundled[-1]
    scratch_length = len(scratch)
    
    rr, cc, val = line_aa(x0, y0, x1, y1)
    
    img[rr,cc] = 1
    
    
plt.imshow(img)

# INCEPTION VISUALIZATION TIME

In [ ]:
Inception_mean_iters= 53902
Inception_std_iters= 54473
Inception_median_iters= 35600

ResNet_mean_iters= 53139
ResNet_std_iters= 60382
ResNet_median_iters= 32480

VGG_mean_iters= 56344
VGG_std_iters= 60843
VGG_median_iters= 36320

Inception_params = 23.8e6
ResNet_params = 25.6e6
VGG_params = 144e6

medians = [Inception_median_iters, ResNet_median_iters,   VGG_median_iters]
means = [Inception_mean_iters, ResNet_mean_iters, Inception_params]

params = [Inception_params, ResNet_params,   VGG_params]

text_array = ['Inception-v3', 'ResNet-50', 'VGG-19']

In [ ]:

fig, ax = plt.subplots(subplot_kw=dict(aspect='equal'))

style = dict(size=10, color='gray')

line, = ax.plot(params, means, '--x', c='r', label='Mean queries')
plt.xlabel('Mean queries')
plt.ylabel('Parameters')


# L2 NORM OF SCRATCHES

In [ ]:
import os
import numpy
import matplotlib.pyplot as plt
import re
import numpy as np
import Image
import pickle
import pretrainedmodels.utils as utils
from skimage.draw import line_aa
import pretrainedmodels
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import requests

LABELS_URL = 'https://s3.amazonaws.com/mlpipes/pytorch-quick-start/labels.json'

labels = {int(key):value for (key, value)
          in requests.get(LABELS_URL).json().items()}

img_directory = '/home/malhar/Desktop/data/imagenet_validation_images/'

model_names = ['inceptionv3', 'resnet18', 'squeezenet1_0', 'squeezenet1_1', 'resnet50', 'vgg11_bn', 'vgg19', 'vgg16', 'resnet34', 'vgg13', 
           'densenet201', 'vgg13_bn', 'resnet152', 'vgg16_bn', 'resnet101', 
               'vgg19_bn', 'vgg11', 'alexnet']

load_img = utils.LoadImage()


data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

def prep_image_for_inference(img_path):
    input_img = load_img(img_path)
    input_tensor = tf_img(input_img)
    input_tensor = input_tensor.unsqueeze(0)
    input_to_model = torch.autograd.Variable(input_tensor,
            requires_grad=False)
#     input_to_model -= torch.min(input_to_model)
#     input_to_model = input_to_model / torch.max(input_to_model)
    return input_to_model


def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def get_probabilities(logits):
    logits_numpy = logits.detach().numpy()
    return softmax(logits_numpy[0])

# VGG-19 L2 NORM CALCULATIONS + PREDICTIONS

In [ ]:

# Get the required folder and the pickled files it
cwd = os.getcwd()
resnet50_dir = cwd + '/' + 'pickled_vgg19/'
resnet50_pickles = os.listdir(resnet50_dir)

model_name = 'vgg19'

l2_norms = []

for pickled_filename in resnet50_pickles:
    # Splice filename into x0,y0,x1,y1 and original and target labels
    print (pickled_filename)
    list_of_split_filename = pickled_filename.split('_')
    source_image_path = list_of_split_filename[0] + '_' + list_of_split_filename[1] + '_' + list_of_split_filename[2]
    target_label = int(list_of_split_filename[3].split('target=')[1])
    original_label = int(list_of_split_filename[4].split('original=')[1])
    x0 = int(list_of_split_filename[6])
    y0 = int(list_of_split_filename[8])
    x1 = int(list_of_split_filename[10])
    y1 = int(list_of_split_filename[12].split('.p')[0])
    print (x0, y0, x1, y1, original_label, target_label)
#     print ('\n')

    # Read in each of the pickles
    pickleFile = open(resnet50_dir + pickled_filename, 'rb')
    adversarial_scratch = pickle.load(pickleFile)
    pickleFile.close()

    # Make the scratch
    rr, cc, val = line_aa(x0, y0, x1, y1)

    # Obtain the model
    model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    model.eval()
    tf_img = utils.TransformImage(model)

    # Load the input to the model
    input_to_model = prep_image_for_inference(img_directory + source_image_path)

#         print (input_to_model.shape)
    if (input_to_model.shape[1] != 3):
        continue

    original_prediction = get_probabilities(model(input_to_model))
    
    input_to_model_original = input_to_model[:]
    
    top_original_prediction = int(np.argmax(original_prediction))
    
#         print ('Target: ', target_label, 'Original: ', original_label)
    print 'Prediction before adding the scratch: ', top_original_prediction
    print 'Label before adding the scratch: ', labels[top_original_prediction]

    # Add the scratch here to the image
    size= len(adversarial_scratch)
    r_vector = torch.from_numpy(adversarial_scratch[0:size/3])
    r_vector = r_vector.type(torch.float)

    g_vector = torch.from_numpy(adversarial_scratch[size/3: int(2*size/3)])
    g_vector = g_vector.type(torch.float)

    b_vector = torch.from_numpy(adversarial_scratch[int(2*size/3): size])
    b_vector = b_vector.type(torch.float)

    input_to_model[0,0,rr,cc] = r_vector
    input_to_model[0,1,rr,cc] = g_vector
    input_to_model[0,2,rr,cc] = b_vector

    input_to_model_adversarial = input_to_model[:]
    
    adversarial_prediction = get_probabilities(model(input_to_model))
    
    top_adversarial_prediction = int(np.argmax(adversarial_prediction))
    
    print 'Prediction after adding the scratch: %d' % top_adversarial_prediction
    print 'Label after adding the scratch: %s' % labels[top_adversarial_prediction]
    print 'Target label: %s' % labels[target_label]
#         print ('\n')
    
    print 'L2 norm %f \n\n' %LA.norm(adversarial_scratch)
    l2_norms.append(LA.norm(adversarial_scratch))
    print 'Mean L2 norm %f \n\n' %np.mean(l2_norms)
    
print (model_name, ', Target: ', target_hits, ', Original: ', original_hits, ', Misclassified: ', misclassified_hits)
print ('\n')

# RESNET-50 L2 NORM CALCULATIONS + PREDICTIONS

In [ ]:

# Get the required folder and the pickled files it
cwd = os.getcwd()
resnet50_dir = cwd + '/' + 'pickled_resnet50/'
resnet50_pickles = os.listdir(resnet50_dir)

l2_norms = []

model_name = 'resnet50'

for pickled_filename in resnet50_pickles:
    # Splice filename into x0,y0,x1,y1 and original and target labels
    print (pickled_filename)
    list_of_split_filename = pickled_filename.split('_')
    source_image_path = list_of_split_filename[0] + '_' + list_of_split_filename[1] + '_' + list_of_split_filename[2]
    target_label = int(list_of_split_filename[3].split('target=')[1])
    original_label = int(list_of_split_filename[4].split('original=')[1])
    x0 = int(list_of_split_filename[6])
    y0 = int(list_of_split_filename[8])
    x1 = int(list_of_split_filename[10])
    y1 = int(list_of_split_filename[12].split('.p')[0])
    print (x0, y0, x1, y1, original_label, target_label)
#     print ('\n')

    # Read in each of the pickles
    pickleFile = open(resnet50_dir + pickled_filename, 'rb')
    adversarial_scratch = pickle.load(pickleFile)
    pickleFile.close()

    # Make the scratch
    rr, cc, val = line_aa(x0, y0, x1, y1)

    # Obtain the model
    model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    model.eval()
    tf_img = utils.TransformImage(model)

    # Load the input to the model
    input_to_model = prep_image_for_inference(img_directory + source_image_path)

#         print (input_to_model.shape)
    if (input_to_model.shape[1] != 3):
        continue

    original_prediction = get_probabilities(model(input_to_model))
    
    input_to_model_original = input_to_model[:]
    
    top_original_prediction = int(np.argmax(original_prediction))
    
#         print ('Target: ', target_label, 'Original: ', original_label)
    print 'Prediction before adding the scratch: ', top_original_prediction
    print 'Label before adding the scratch: ', labels[top_original_prediction]

    # Add the scratch here to the image
    size= len(adversarial_scratch)
    r_vector = torch.from_numpy(adversarial_scratch[0:size/3])
    r_vector = r_vector.type(torch.float)

    g_vector = torch.from_numpy(adversarial_scratch[size/3: int(2*size/3)])
    g_vector = g_vector.type(torch.float)

    b_vector = torch.from_numpy(adversarial_scratch[int(2*size/3): size])
    b_vector = b_vector.type(torch.float)

    input_to_model[0,0,rr,cc] = r_vector
    input_to_model[0,1,rr,cc] = g_vector
    input_to_model[0,2,rr,cc] = b_vector

    input_to_model_adversarial = input_to_model[:]
    
    adversarial_prediction = get_probabilities(model(input_to_model))
    
    top_adversarial_prediction = int(np.argmax(adversarial_prediction))
    
    print 'Prediction after adding the scratch: %d' % top_adversarial_prediction
    print 'Label after adding the scratch: %s' % labels[top_adversarial_prediction]
    print 'Target label: %s' % labels[target_label]
#         print ('\n')
    print 'L2 norm %f \n\n' %LA.norm(adversarial_scratch)
    l2_norms.append(LA.norm(adversarial_scratch))
    print 'Mean L2 norm %f \n\n' %np.mean(l2_norms)
    
#     if (torch.argmax(original_prediction) == original_label):
#         correct += 1
#     else:
#         incorrect += 1

#     if (torch.argmax(adversarial_prediction) == target_label):
#         target_hits += 1

#     elif (torch.argmax(adversarial_prediction) == original_label):
#         original_hits += 1

#     else:
#         misclassified_hits += 1
print (model_name, ', Target: ', target_hits, ', Original: ', original_hits, ', Misclassified: ', misclassified_hits)
print ('\n')

# INCEPTION-V3 L2 NORM CALCULATIONS

In [ ]:

# Get the required folder and the pickled files it
cwd = os.getcwd()
resnet50_dir = cwd + '/' + 'pickled_image_parts_abs/'
resnet50_pickles = os.listdir(resnet50_dir)

l2_norms = []

model_name = 'inceptionv3'

for pickled_filename in resnet50_pickles:
    pickleFile = open(resnet50_dir + pickled_filename, 'rb')
    adversarial_scratch = pickle.load(pickleFile)
    pickleFile.close()

    print 'L2 norm %f \n\n' %LA.norm(adversarial_scratch)
    l2_norms.append(LA.norm(adversarial_scratch))
    print 'Mean L2 norm %f \n\n' %np.mean(l2_norms)

# VGG-19 L2 NORM + ITERATIONS REQUIRED

In [ ]:
################################################################################################
################################################################################################
################################################################################################
# VGG ##########################################################################################

# Get the required folder and the pickled files it
cwd = os.getcwd()
PICKLE_DIRECTORY = cwd + '/' + 'pickled_vgg19/'
PICKLES = os.listdir(PICKLE_DIRECTORY)

LOG_DIRECTORY = cwd + '/' + 'log_vgg19/'
LOGS = os.listdir(LOG_DIRECTORY)

IMSIZE_VGG_RESNET = 244*244*3
IMSIZE_INCEPTION = 299*299*3


model_name = 'vgg19'

vgg_l2_norms = []
vgg_iterations_required = []
vgg_scratch_sizes = []

for pickled_filename in PICKLES:
    # Splice filename into x0,y0,x1,y1 and original and target labels
#     print (pickled_filename)# Read in each of the pickles
    
    list_of_split_filename = pickled_filename.split('_')
    source_image_path = list_of_split_filename[0] + '_' + list_of_split_filename[1] + '_' + list_of_split_filename[2]
    target_label = (list_of_split_filename[3].split('target=')[1])
    original_label = (list_of_split_filename[4].split('original=')[1])
    x0 = (list_of_split_filename[6])
    y0 = (list_of_split_filename[8])
    x1 = (list_of_split_filename[10])
    y1 = (list_of_split_filename[12].split('.p')[0])
    
    # Read in the pickle
    pickleFile = open(PICKLE_DIRECTORY + pickled_filename, 'rb')
    adversarial_scratch = pickle.load(pickleFile)
    pickleFile.close()
    vgg_scratch_sizes.append(100. * len(adversarial_scratch)/IMSIZE_VGG_RESNET)

    # remake the string
    fname = [s for s in LOGS if (source_image_path in s and x0 in s and y0 in s and x1 in s and y1 in s and target_label in s and original_label in s)]
    fname = fname[0]
    
    with open(LOG_DIRECTORY + fname) as f:
        data = f.readlines()
        for line in data:
            if ('Reached' in line):
                iters = [int(s) for s in line.split() if s.isdigit()]
                vgg_iterations_required.append(iters[0])
    
#     print 'L2 norm %f' %LA.norm(adversarial_scratch)
    vgg_l2_norms.append(LA.norm(adversarial_scratch))
#     print 'Mean L2 norm %f \n' %np.mean(vgg_l2_norms)
    
################################################################################################
################################################################################################
################################################################################################
# RESNET########################################################################################
# Get the required folder and the pickled files it
cwd = os.getcwd()
PICKLE_DIRECTORY = cwd + '/' + 'pickled_resnet50/'
PICKLES = os.listdir(PICKLE_DIRECTORY)

LOG_DIRECTORY = cwd + '/' + 'log_resnet50/'
LOGS = os.listdir(LOG_DIRECTORY)


model_name = 'resnet50'

resnet_l2_norms = []
resnet_iterations_required = []
resnet_scratch_sizes = []

for pickled_filename in PICKLES:
    # Splice filename into x0,y0,x1,y1 and original and target labels
#     print (pickled_filename)# Read in each of the pickles
    
    list_of_split_filename = pickled_filename.split('_')
    source_image_path = list_of_split_filename[0] + '_' + list_of_split_filename[1] + '_' + list_of_split_filename[2]
    target_label = (list_of_split_filename[3].split('target=')[1])
    original_label = (list_of_split_filename[4].split('original=')[1])
    x0 = (list_of_split_filename[6])
    y0 = (list_of_split_filename[8])
    x1 = (list_of_split_filename[10])
    y1 = (list_of_split_filename[12].split('.p')[0])
    
    # Read in the pickle
    pickleFile = open(PICKLE_DIRECTORY + pickled_filename, 'rb')
    adversarial_scratch = pickle.load(pickleFile)
    pickleFile.close()
    resnet_scratch_sizes.append(100.* len(adversarial_scratch)/IMSIZE_VGG_RESNET)
    
    # remake the string
    fname = [s for s in LOGS if (source_image_path in s and x0 in s and y0 in s and x1 in s and y1 in s and target_label in s and original_label in s)]
    fname = fname[0]
    
    with open(LOG_DIRECTORY + fname) as f:
        data = f.readlines()
        for line in data:
            if ('Reached' in line):
                iters = [int(s) for s in line.split() if s.isdigit()]
                resnet_iterations_required.append(iters[0])
    
#     print 'L2 norm %f' %LA.norm(adversarial_scratch)
    resnet_l2_norms.append(LA.norm(adversarial_scratch))
#     print 'Mean L2 norm %f \n' %np.mean(resnet_l2_norms)
    

    
################################################################################################
################################################################################################
################################################################################################
# INCEPTION ####################################################################################

cwd = os.getcwd()
PICKLE_DIRECTORY = cwd + '/' + 'pickled_image_parts_abs/'
PICKLES = os.listdir(PICKLE_DIRECTORY)

LOG_DIRECTORY = cwd + '/' + 'log_abs/'
LOGS = os.listdir(LOG_DIRECTORY)


model_name = 'inceptionv3'

inception_l2_norms = []
inception_iterations_required = []
inception_scratch_sizes = []


for pickled_filename in PICKLES:
    # Splice filename into x0,y0,x1,y1 and original and target labels
    print (pickled_filename)# Read in each of the pickles
    
    list_of_split_filename = pickled_filename.split('_')
    print list_of_split_filename
    target_label = (list_of_split_filename[0].split('target=')[1])
    original_label = (list_of_split_filename[1].split('original=')[1])
    x0 = (list_of_split_filename[3])
    y0 = (list_of_split_filename[5])
    x1 = (list_of_split_filename[7])
    y1 = (list_of_split_filename[9].split('.p')[0])
    
    # Read in the pickle
    pickleFile = open(PICKLE_DIRECTORY + pickled_filename, 'rb')
    adversarial_scratch = pickle.load(pickleFile)
    pickleFile.close()
    inception_scratch_sizes.append(100. * len(adversarial_scratch) / IMSIZE_INCEPTION)
    
    # remake the string
    fname = [s for s in LOGS if (x0 in s and y0 in s and x1 in s and y1 in s and target_label in s and original_label in s)]
    print fname
    fname = fname[0]
    
    with open(LOG_DIRECTORY + fname) as f:
        data = f.readlines()
        for line in data:
            if ('Reached' in line):
                iters = [int(s) for s in line.split() if s.isdigit()]
                inception_iterations_required.append(iters[0])
    
    print 'L2 norm %f' %LA.norm(adversarial_scratch)
    inception_l2_norms.append(LA.norm(adversarial_scratch))
    print 'Mean L2 norm %f \n' %np.mean(inception_l2_norms)
    

    

In [ ]:
################################################################################################
################################################################################################
################################################################################################
# VGG ##########################################################################################
fig = plt.figure()
ax = plt.gca()
ax.scatter(vgg_iterations_required, vgg_l2_norms)
plt.xlabel('Populations Required', fontsize=15)
plt.ylabel('L2 norm of scratch', fontsize=15)
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15) 
plt.title('VGG-19')

fig = plt.figure()
ax = plt.gca()
ax.scatter(vgg_iterations_required, vgg_l2_norms, c='r')
plt.xlabel('Populations Required', fontsize=15)
plt.ylabel('L2 norm of scratch', fontsize=15)
# ax.set_yscale('log')
ax.set_xscale('log')
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15) 
plt.title('VGG-19')

################################################################################################
################################################################################################
################################################################################################
# RESNET #######################################################################################
fig = plt.figure()
ax = plt.gca()
ax.scatter(resnet_iterations_required, resnet_l2_norms)
plt.xlabel('Populations Required', fontsize=15)
plt.ylabel('L2 norm of scratch', fontsize=15)
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15) 
plt.title('ResNet-50')

fig = plt.figure()
ax = plt.gca()
ax.scatter(resnet_iterations_required, resnet_l2_norms, c='r')
plt.xlabel('Populations Required', fontsize=15)
plt.ylabel('L2 norm of scratch', fontsize=15)
# ax.set_yscale('log')
ax.set_xscale('log')
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15) 
plt.title('ResNet-50')

################################################################################################
################################################################################################
################################################################################################
# INCEPTION ####################################################################################
fig = plt.figure()
ax = plt.gca()
ax.scatter(inception_iterations_required, inception_l2_norms)
plt.xlabel('Populations Required', fontsize=15)
plt.ylabel('L2 norm of scratch', fontsize=15)
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15) 
plt.title('Inception-v3')

fig = plt.figure()
ax = plt.gca()
ax.scatter(inception_iterations_required, inception_l2_norms, c='r')
plt.xlabel('Populations Required', fontsize=15)
plt.ylabel('L2 norm of scratch', fontsize=15)
# ax.set_yscale('log')
ax.set_xscale('log')
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15) 
plt.title('Inception-v3')

################################################################################################
################################################################################################
################################################################################################
# ALL THREE ####################################################################################
fig = plt.figure()
ax = plt.gca()
ax.scatter(resnet_iterations_required, resnet_l2_norms, c='r', label='ResNet-50')
ax.scatter(vgg_iterations_required, vgg_l2_norms, c='g', label='VGG-19')
ax.scatter(inception_iterations_required, inception_l2_norms, c='k', label='Inception-v3')
plt.xlabel('Populations Required', fontsize=15)
plt.ylabel('L2 norm of scratch', fontsize=15)
# ax.set_yscale('log')
ax.set_xscale('log')
plt.rc('xtick', labelsize=15) 
plt.rc('ytick', labelsize=15) 
plt.title('L2 norm v/s populations of successful scratches', size=15)
plt.legend(prop={'size': 15})

In [ ]:
################################################################################################
################################################################################################
################################################################################################
# ALL THREE ####################################################################################
fig = plt.figure(figsize=(8, 8))
ax = plt.gca()
SIZE = 25
ax.scatter(resnet_iterations_required, resnet_l2_norms, c='r', label='ResNet-50')
ax.scatter(vgg_iterations_required, vgg_l2_norms, c='g', label='VGG-19')
ax.scatter(inception_iterations_required, inception_l2_norms, c='k', label='Inception-v3')
plt.xlabel('Populations Required', fontsize=SIZE)
plt.ylabel('L2 norm of scratch', fontsize=SIZE)
ax.set_yscale('log')
ax.set_xscale('log')
plt.rc('xtick', labelsize=SIZE) 
plt.rc('ytick', labelsize=SIZE) 
plt.title('L2 norm v/s population size', size=SIZE)
plt.legend(prop={'size': SIZE})
fig.tight_layout()
plt.savefig('images/l2_vs_populations.pdf', format='pdf')

################################################################################################
################################################################################################
################################################################################################
# ALL THREE ####################################################################################
fig = plt.figure(figsize=(8, 8))
ax = plt.gca()
ax.scatter(resnet_iterations_required, resnet_scratch_sizes, c='r', label='ResNet-50')
ax.scatter(vgg_iterations_required, vgg_scratch_sizes, c='g', label='VGG-19')
ax.scatter(inception_iterations_required, inception_scratch_sizes, c='k', label='Inception-v3')
plt.xlabel('Populations Required', fontsize=SIZE)
plt.ylabel('Scratch size', fontsize=SIZE)
# ax.set_yscale('log')
ax.set_xscale('log')
plt.rc('xtick', labelsize=SIZE) 
plt.rc('ytick', labelsize=SIZE) 
plt.title('Scratch size v/s population size', size=SIZE)
plt.legend(prop={'size': SIZE})
fig.tight_layout()
plt.savefig('images/scratch_size_vs_populations.pdf', format='pdf')


# TRANSFERABILITY OF SCRATCHES: RESNET GOING TO OTHER MODELS

In [28]:
model_names = ['inceptionv3', 'resnet18', 'squeezenet1_0', 'squeezenet1_1', 'resnet50', 'vgg19', 'vgg16', 'resnet34', 'vgg13', 
               'densenet201', 'resnet152', 'resnet101', 'vgg11', 'alexnet']

# Get the required folder and the pickled files it
cwd = os.getcwd()
resnet50_dir = cwd + '/' + 'pickled_vgg19/'
resnet50_pickles = os.listdir(resnet50_dir)

l2_norms = []

original_model_name = 'vgg19'

for model_name in model_names:
    
    if (model_name == original_model_name):
        continue
    
    # Obtain the model
    model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    model.eval()
    tf_img = utils.TransformImage(model)
    
    correct = 0
    incorrect = 0
    target_hits = 0
    original_hits = 0
    misclassified_hits = 0
    
    for iterz, pickled_filename in zip(range(len(resnet50_pickles)), resnet50_pickles):
        
        # Splice filename into x0,y0,x1,y1 and original and target labels
#         print (pickled_filename)
        list_of_split_filename = pickled_filename.split('_')
        source_image_path = list_of_split_filename[0] + '_' + list_of_split_filename[1] + '_' + list_of_split_filename[2]
        target_label = int(list_of_split_filename[3].split('target=')[1])
        original_label = int(list_of_split_filename[4].split('original=')[1])
        x0 = int(list_of_split_filename[6])
        y0 = int(list_of_split_filename[8])
        x1 = int(list_of_split_filename[10])
        y1 = int(list_of_split_filename[12].split('.p')[0])
#         print (x0, y0, x1, y1, original_label, target_label)
    #     print ('\n')

        # Read in each of the pickles
        pickleFile = open(resnet50_dir + pickled_filename, 'rb')
        adversarial_scratch = pickle.load(pickleFile)
        pickleFile.close()

        # Make the scratch
        rr, cc, val = line_aa(x0, y0, x1, y1)

        # Load the input to the model
        input_to_model = prep_image_for_inference(img_directory + source_image_path)

    #         print (input_to_model.shape)
        if (input_to_model.shape[1] != 3):
            continue

        original_prediction = get_probabilities(model(input_to_model))

        input_to_model_original = input_to_model[:]

        top_original_prediction = int(np.argmax(original_prediction))
        
        # Dont want to skew up our results with already misclassified images
        if (top_original_prediction != original_label):
            continue
        else:
            correct += 1

    #         print ('Target: ', target_label, 'Original: ', original_label)
        print correct, 'Prediction before adding the scratch: ', top_original_prediction
#         print 'Label before adding the scratch: ', labels[top_original_prediction]

        # Add the scratch here to the image
        size= len(adversarial_scratch)
        r_vector = torch.from_numpy(adversarial_scratch[0:size/3])
        r_vector = r_vector.type(torch.float)

        g_vector = torch.from_numpy(adversarial_scratch[size/3: int(2*size/3)])
        g_vector = g_vector.type(torch.float)

        b_vector = torch.from_numpy(adversarial_scratch[int(2*size/3): size])
        b_vector = b_vector.type(torch.float)

        input_to_model[0,0,rr,cc] = r_vector
        input_to_model[0,1,rr,cc] = g_vector
        input_to_model[0,2,rr,cc] = b_vector

        input_to_model_adversarial = input_to_model[:]

        adversarial_prediction = get_probabilities(model(input_to_model))

        top_adversarial_prediction = int(np.argmax(adversarial_prediction))

        print 'Prediction after adding the scratch: %d' % top_adversarial_prediction
#         print 'Label after adding the scratch: %s' % labels[top_adversarial_prediction]
#         print 'Target label: %s' % labels[target_label]
#         print ('\n')

#         if (top_original_prediction == original_label):
#             correct += 1
#         else:
#             incorrect += 1
        if (top_adversarial_prediction == target_label):
            target_hits += 1
        elif (top_adversarial_prediction == original_label):
            original_hits += 1
        else:
            misclassified_hits += 1
            
        if (correct >= 50):
            break
    
    print (model_name, ', Target: ', target_hits, ', Original: ', original_hits, ', Misclassified: ', misclassified_hits)
    print ('\n')

1 Prediction before adding the scratch:  853
Prediction after adding the scratch: 853
2 Prediction before adding the scratch:  243
Prediction after adding the scratch: 243
3 Prediction before adding the scratch:  89
Prediction after adding the scratch: 89
4 Prediction before adding the scratch:  302
Prediction after adding the scratch: 916
5 Prediction before adding the scratch:  442
Prediction after adding the scratch: 916
6 Prediction before adding the scratch:  846
Prediction after adding the scratch: 804
7 Prediction before adding the scratch:  205
Prediction after adding the scratch: 701
8 Prediction before adding the scratch:  302
Prediction after adding the scratch: 916
9 Prediction before adding the scratch:  195
Prediction after adding the scratch: 195
10 Prediction before adding the scratch:  367
Prediction after adding the scratch: 378
11 Prediction before adding the scratch:  559
Prediction after adding the scratch: 872
12 Prediction before adding the scratch:  125
Predicti

47 Prediction before adding the scratch:  353
Prediction after adding the scratch: 352
48 Prediction before adding the scratch:  63
Prediction after adding the scratch: 63
49 Prediction before adding the scratch:  322
Prediction after adding the scratch: 322
50 Prediction before adding the scratch:  486
Prediction after adding the scratch: 486
('resnet18', ', Target: ', 0, ', Original: ', 28, ', Misclassified: ', 22)


1 Prediction before adding the scratch:  853
Prediction after adding the scratch: 972
2 Prediction before adding the scratch:  243
Prediction after adding the scratch: 243
3 Prediction before adding the scratch:  89
Prediction after adding the scratch: 693
4 Prediction before adding the scratch:  302
Prediction after adding the scratch: 470
5 Prediction before adding the scratch:  353
Prediction after adding the scratch: 971
6 Prediction before adding the scratch:  846
Prediction after adding the scratch: 736
7 Prediction before adding the scratch:  302
Prediction after 

40 Prediction before adding the scratch:  740
Prediction after adding the scratch: 795
41 Prediction before adding the scratch:  337
Prediction after adding the scratch: 920
42 Prediction before adding the scratch:  991
Prediction after adding the scratch: 971
43 Prediction before adding the scratch:  561
Prediction after adding the scratch: 569
44 Prediction before adding the scratch:  337
Prediction after adding the scratch: 321
45 Prediction before adding the scratch:  800
Prediction after adding the scratch: 920
46 Prediction before adding the scratch:  89
Prediction after adding the scratch: 920
47 Prediction before adding the scratch:  195
Prediction after adding the scratch: 815
48 Prediction before adding the scratch:  540
Prediction after adding the scratch: 510
49 Prediction before adding the scratch:  205
Prediction after adding the scratch: 664
50 Prediction before adding the scratch:  652
Prediction after adding the scratch: 795
('squeezenet1_1', ', Target: ', 0, ', Origin

34 Prediction before adding the scratch:  606
Prediction after adding the scratch: 306
35 Prediction before adding the scratch:  1
Prediction after adding the scratch: 1
36 Prediction before adding the scratch:  258
Prediction after adding the scratch: 258
37 Prediction before adding the scratch:  929
Prediction after adding the scratch: 813
38 Prediction before adding the scratch:  195
Prediction after adding the scratch: 971
39 Prediction before adding the scratch:  322
Prediction after adding the scratch: 321
40 Prediction before adding the scratch:  800
Prediction after adding the scratch: 800
41 Prediction before adding the scratch:  606
Prediction after adding the scratch: 512
42 Prediction before adding the scratch:  302
Prediction after adding the scratch: 302
43 Prediction before adding the scratch:  104
Prediction after adding the scratch: 104
44 Prediction before adding the scratch:  285
Prediction after adding the scratch: 285
45 Prediction before adding the scratch:  846
P

Prediction after adding the scratch: 920
29 Prediction before adding the scratch:  285
Prediction after adding the scratch: 87
30 Prediction before adding the scratch:  89
Prediction after adding the scratch: 89
31 Prediction before adding the scratch:  353
Prediction after adding the scratch: 971
32 Prediction before adding the scratch:  606
Prediction after adding the scratch: 679
33 Prediction before adding the scratch:  1
Prediction after adding the scratch: 1
34 Prediction before adding the scratch:  258
Prediction after adding the scratch: 258
35 Prediction before adding the scratch:  929
Prediction after adding the scratch: 731
36 Prediction before adding the scratch:  195
Prediction after adding the scratch: 417
37 Prediction before adding the scratch:  322
Prediction after adding the scratch: 325
38 Prediction before adding the scratch:  800
Prediction after adding the scratch: 800
39 Prediction before adding the scratch:  606
Prediction after adding the scratch: 907
40 Predic

22 Prediction before adding the scratch:  195
Prediction after adding the scratch: 876
23 Prediction before adding the scratch:  321
Prediction after adding the scratch: 326
24 Prediction before adding the scratch:  62
Prediction after adding the scratch: 819
25 Prediction before adding the scratch:  821
Prediction after adding the scratch: 821
26 Prediction before adding the scratch:  1
Prediction after adding the scratch: 1
27 Prediction before adding the scratch:  337
Prediction after adding the scratch: 337
28 Prediction before adding the scratch:  855
Prediction after adding the scratch: 494
29 Prediction before adding the scratch:  606
Prediction after adding the scratch: 528
30 Prediction before adding the scratch:  7
Prediction after adding the scratch: 8
31 Prediction before adding the scratch:  285
Prediction after adding the scratch: 285
32 Prediction before adding the scratch:  942
Prediction after adding the scratch: 942
33 Prediction before adding the scratch:  89
Predict

Prediction after adding the scratch: 89
16 Prediction before adding the scratch:  285
Prediction after adding the scratch: 285
17 Prediction before adding the scratch:  205
Prediction after adding the scratch: 205
18 Prediction before adding the scratch:  314
Prediction after adding the scratch: 314
19 Prediction before adding the scratch:  17
Prediction after adding the scratch: 17
20 Prediction before adding the scratch:  195
Prediction after adding the scratch: 892
21 Prediction before adding the scratch:  321
Prediction after adding the scratch: 815
22 Prediction before adding the scratch:  846
Prediction after adding the scratch: 619
23 Prediction before adding the scratch:  62
Prediction after adding the scratch: 971
24 Prediction before adding the scratch:  821
Prediction after adding the scratch: 733
25 Prediction before adding the scratch:  1
Prediction after adding the scratch: 723
26 Prediction before adding the scratch:  337
Prediction after adding the scratch: 224
27 Predi

In [ ]:
model_names = ['inceptionv3', 'resnet18', 'squeezenet1_0', 'squeezenet1_1', 'resnet50', 'vgg19', 'vgg16', 'resnet34', 'vgg13', 
               'densenet201', 'resnet152', 'resnet101', 'vgg11', 'alexnet']

# Get the required folder and the pickled files it
cwd = os.getcwd()
resnet50_dir = cwd + '/' + 'pickled_vgg19/'
resnet50_pickles = os.listdir(resnet50_dir)

l2_norms = []

original_model_name = 'vgg19'

for model_name in model_names:
    
    if (model_name == original_model_name):
        continue
    
    # Obtain the model
    model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    model.eval()
    tf_img = utils.TransformImage(model)
    
    correct = 0
    incorrect = 0
    target_hits = 0
    original_hits = 0
    misclassified_hits = 0
    
    for pickled_filename in resnet50_pickles:
        # Splice filename into x0,y0,x1,y1 and original and target labels
        print 'Attacking %s' %model_name
#         print (pickled_filename)
        list_of_split_filename = pickled_filename.split('_')
        source_image_path = list_of_split_filename[0] + '_' + list_of_split_filename[1] + '_' + list_of_split_filename[2]
        target_label = int(list_of_split_filename[3].split('target=')[1])
        original_label = int(list_of_split_filename[4].split('original=')[1])
        x0 = int(list_of_split_filename[6])
        y0 = int(list_of_split_filename[8])
        x1 = int(list_of_split_filename[10])
        y1 = int(list_of_split_filename[12].split('.p')[0])
#         print (x0, y0, x1, y1, original_label, target_label)
    #     print ('\n')

        # Read in each of the pickles
        pickleFile = open(resnet50_dir + pickled_filename, 'rb')
        adversarial_scratch = pickle.load(pickleFile)
        pickleFile.close()

        # Make the scratch
        rr, cc, val = line_aa(x0, y0, x1, y1)

        # Load the input to the model
        input_to_model = prep_image_for_inference(img_directory + source_image_path)

    #         print (input_to_model.shape)
        if (input_to_model.shape[1] != 3):
            continue

        original_prediction = get_probabilities(model(input_to_model))

        input_to_model_original = input_to_model[:]

        top_original_prediction = int(np.argmax(original_prediction))

    #         print ('Target: ', target_label, 'Original: ', original_label)
        print 'Prediction before adding the scratch: ', top_original_prediction
#         print 'Label before adding the scratch: ', labels[top_original_prediction]

        # Add the scratch here to the image
        size= len(adversarial_scratch)
        r_vector = torch.from_numpy(adversarial_scratch[0:size/3])
        r_vector = r_vector.type(torch.float)

        g_vector = torch.from_numpy(adversarial_scratch[size/3: int(2*size/3)])
        g_vector = g_vector.type(torch.float)

        b_vector = torch.from_numpy(adversarial_scratch[int(2*size/3): size])
        b_vector = b_vector.type(torch.float)

        input_to_model[0,0,rr,cc] = r_vector
        input_to_model[0,1,rr,cc] = g_vector
        input_to_model[0,2,rr,cc] = b_vector

        input_to_model_adversarial = input_to_model[:]

        adversarial_prediction = get_probabilities(model(input_to_model))

        top_adversarial_prediction = int(np.argmax(adversarial_prediction))

        print 'Prediction after adding the scratch: %d' % top_adversarial_prediction
#         print 'Label after adding the scratch: %s' % labels[top_adversarial_prediction]
#         print 'Target label: %s' % labels[target_label]
#         print ('\n')

        if (top_original_prediction == original_label):
            correct += 1
        else:
            incorrect += 1
        if (top_adversarial_prediction == target_label):
            target_hits += 1
        elif (top_adversarial_prediction == original_label):
            original_hits += 1
        else:
            misclassified_hits += 1
    print (model_name, ', Target: ', target_hits, ', Original: ', original_hits, ', Misclassified: ', misclassified_hits)
    print ('\n')

In [21]:
model_names = ['inceptionv3', 'resnet18', 'squeezenet1_0', 'squeezenet1_1', 'resnet50', 'vgg19', 'vgg16', 'resnet34', 'vgg13', 
               'densenet201', 'resnet152', 'resnet101', 'vgg11', 'alexnet']

# Get the required folder and the pickled files it
cwd = os.getcwd()
resnet50_dir = cwd + '/' + 'pickled_vgg19/'
resnet50_pickles = os.listdir(resnet50_dir)

l2_norms = []

original_model_name = 'vgg19'

for model_name in model_names:
    
#     if (model_name != original_model_name):
#         continue
    
    # Obtain the model
    model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')
    model.eval()
    tf_img = utils.TransformImage(model)
    
    correct = 0
    incorrect = 0
    target_hits = 0
    original_hits = 0
    misclassified_hits = 0
    
    correct_preds = set()
    incorrect_preds = set()
    
    for pickled_filename in resnet50_pickles:
        
        if (len(correct_preds) + len(incorrect_preds) > 100):
            break
        
        # Splice filename into x0,y0,x1,y1 and original and target labels
        print 'Attacking %s' %model_name
#         print (pickled_filename)
        list_of_split_filename = pickled_filename.split('_')
        source_image_path = list_of_split_filename[0] + '_' + list_of_split_filename[1] + '_' + list_of_split_filename[2]
        target_label = int(list_of_split_filename[3].split('target=')[1])
        original_label = int(list_of_split_filename[4].split('original=')[1])

        # Load the input to the model
        input_to_model = prep_image_for_inference(img_directory + source_image_path)

    #         print (input_to_model.shape)
        if (input_to_model.shape[1] != 3):
            continue

        original_prediction = get_probabilities(model(input_to_model))

        input_to_model_original = input_to_model[:]

        top_original_prediction = int(np.argmax(original_prediction))

    #         print ('Target: ', target_label, 'Original: ', original_label)
        print 'Prediction before adding the scratch: %d, Original class %d' %(top_original_prediction, original_label)
#         print 'Label before adding the scratch: ', labels[top_original_prediction]
        try:
            print 1.0 * len(correct_preds) / (len(incorrect_preds) + len(correct_preds))
        except:
            print 'sorry'
    
        if (top_original_prediction == original_label):
            correct += 1
            correct_preds.add(original_label)
        else:
            incorrect += 1
            incorrect_preds.add(top_original_prediction)
    
    print model_name, 'Correct: ', len(correct_preds), 'Incorrect: ', len(incorrect_preds)
#     print (model_name, ', Target: ', target_hits, ', Original: ', original_hits, ', Misclassified: ', misclassified_hits)
    print ('\n')

Attacking inceptionv3
Prediction before adding the scratch: 853, Original class 853
sorry
Attacking inceptionv3
Prediction before adding the scratch: 243, Original class 243
1.0
Attacking inceptionv3
Prediction before adding the scratch: 89, Original class 89
1.0
Attacking inceptionv3
Prediction before adding the scratch: 302, Original class 302
1.0
Attacking inceptionv3
Prediction before adding the scratch: 509, Original class 454
1.0
Attacking inceptionv3
Prediction before adding the scratch: 352, Original class 353
0.8
Attacking inceptionv3
Prediction before adding the scratch: 442, Original class 442
0.666666666667
Attacking inceptionv3
Prediction before adding the scratch: 846, Original class 846
0.714285714286
Attacking inceptionv3
Prediction before adding the scratch: 205, Original class 205
0.75
Attacking inceptionv3
Prediction before adding the scratch: 302, Original class 302
0.777777777778
Attacking inceptionv3
Prediction before adding the scratch: 195, Original class 195
0.

Prediction before adding the scratch: 540, Original class 540
0.795918367347
Attacking inceptionv3
Prediction before adding the scratch: 205, Original class 205
0.8
Attacking inceptionv3
Prediction before adding the scratch: 652, Original class 652
0.8
Attacking inceptionv3
Prediction before adding the scratch: 63, Original class 63
0.803921568627
Attacking inceptionv3
Prediction before adding the scratch: 25, Original class 25
0.803921568627
Attacking inceptionv3
Prediction before adding the scratch: 17, Original class 17
0.807692307692
Attacking inceptionv3
Prediction before adding the scratch: 464, Original class 626
0.807692307692
Attacking inceptionv3
Prediction before adding the scratch: 843, Original class 488
0.792452830189
Attacking inceptionv3
Prediction before adding the scratch: 840, Original class 523
0.777777777778
Attacking inceptionv3
Prediction before adding the scratch: 646, Original class 646
0.777777777778
Attacking inceptionv3
Prediction before adding the scratch: 

Prediction before adding the scratch: 881, Original class 881
0.789473684211
Attacking inceptionv3
Prediction before adding the scratch: 273, Original class 273
0.789473684211
Attacking inceptionv3
Prediction before adding the scratch: 559, Original class 559
0.792207792208
Attacking inceptionv3
Prediction before adding the scratch: 104, Original class 104
0.792207792208
Attacking inceptionv3
Prediction before adding the scratch: 879, Original class 515
0.792207792208
Attacking inceptionv3
Prediction before adding the scratch: 337, Original class 337
0.792207792208
Attacking inceptionv3
Prediction before adding the scratch: 336, Original class 336
0.792207792208
Attacking inceptionv3
Prediction before adding the scratch: 896, Original class 896
0.794871794872
Attacking inceptionv3
Prediction before adding the scratch: 843, Original class 488
0.79746835443
Attacking inceptionv3
Prediction before adding the scratch: 542, Original class 541
0.79746835443
Attacking inceptionv3
Prediction b

Prediction before adding the scratch: 243, Original class 243
0.797752808989
Attacking inceptionv3
Prediction before adding the scratch: 1, Original class 1
0.797752808989
Attacking inceptionv3
Prediction before adding the scratch: 25, Original class 25
0.797752808989
Attacking inceptionv3
Prediction before adding the scratch: 72, Original class 72
0.797752808989
Attacking inceptionv3
Prediction before adding the scratch: 130, Original class 130
0.797752808989
Attacking inceptionv3
Prediction before adding the scratch: 853, Original class 853
0.797752808989
Attacking inceptionv3
Prediction before adding the scratch: 336, Original class 336
0.797752808989
Attacking inceptionv3
Prediction before adding the scratch: 929, Original class 929
0.797752808989
Attacking inceptionv3
Prediction before adding the scratch: 442, Original class 442
0.797752808989
Attacking inceptionv3
Prediction before adding the scratch: 843, Original class 843
0.797752808989
Attacking inceptionv3
Prediction before 

Prediction before adding the scratch: 292, Original class 292
0.791208791209
Attacking inceptionv3
Prediction before adding the scratch: 8, Original class 7
0.791208791209
Attacking inceptionv3
Prediction before adding the scratch: 357, Original class 356
0.791208791209
Attacking inceptionv3
Prediction before adding the scratch: 319, Original class 320
0.791208791209
Attacking inceptionv3
Prediction before adding the scratch: 229, Original class 229
0.782608695652
Attacking inceptionv3
Prediction before adding the scratch: 449, Original class 449
0.782608695652
Attacking inceptionv3
Prediction before adding the scratch: 561, Original class 561
0.782608695652
Attacking inceptionv3
Prediction before adding the scratch: 855, Original class 855
0.782608695652
Attacking inceptionv3
Prediction before adding the scratch: 840, Original class 523
0.782608695652
Attacking inceptionv3
Prediction before adding the scratch: 258, Original class 258
0.782608695652
Attacking inceptionv3
Prediction bef

...

Prediction before adding the scratch: 740, Original class 740
0.666666666667
Attacking alexnet
Prediction before adding the scratch: 721, Original class 523
0.674418604651
Attacking alexnet
Prediction before adding the scratch: 337, Original class 337
0.659090909091
Attacking alexnet
Prediction before adding the scratch: 991, Original class 991
0.659090909091
Attacking alexnet
Prediction before adding the scratch: 559, Original class 559
0.666666666667
Attacking alexnet
Prediction before adding the scratch: 784, Original class 499
0.666666666667
Attacking alexnet
Prediction before adding the scratch: 989, Original class 989
0.666666666667
Attacking alexnet
Prediction before adding the scratch: 561, Original class 561
0.673913043478
Attacking alexnet
Prediction before adding the scratch: 337, Original class 337
0.673913043478
Attacking alexnet
Prediction before adding the scratch: 314, Original class 314
0.673913043478
Attacking alexnet
Prediction before adding the scratch: 800, Origina

Prediction before adding the scratch: 989, Original class 989
0.662162162162
Attacking alexnet
Prediction before adding the scratch: 1, Original class 1
0.662162162162
Attacking alexnet
Prediction before adding the scratch: 179, Original class 179
0.662162162162
Attacking alexnet
Prediction before adding the scratch: 104, Original class 104
0.662162162162
Attacking alexnet
Prediction before adding the scratch: 848, Original class 848
0.662162162162
Attacking alexnet
Prediction before adding the scratch: 410, Original class 410
0.662162162162
Attacking alexnet
Prediction before adding the scratch: 337, Original class 337
0.662162162162
Attacking alexnet
Prediction before adding the scratch: 67, Original class 62
0.662162162162
Attacking alexnet
Prediction before adding the scratch: 302, Original class 302
0.662162162162
Attacking alexnet
Prediction before adding the scratch: 233, Original class 265
0.662162162162
Attacking alexnet
Prediction before adding the scratch: 536, Original clas

Prediction before adding the scratch: 314, Original class 314
0.670454545455
Attacking alexnet
Prediction before adding the scratch: 853, Original class 853
0.670454545455
Attacking alexnet
Prediction before adding the scratch: 784, Original class 499
0.670454545455
Attacking alexnet
Prediction before adding the scratch: 693, Original class 646
0.670454545455
Attacking alexnet
Prediction before adding the scratch: 285, Original class 285
0.670454545455
Attacking alexnet
Prediction before adding the scratch: 721, Original class 523
0.670454545455
Attacking alexnet
Prediction before adding the scratch: 659, Original class 809
0.670454545455
Attacking alexnet
Prediction before adding the scratch: 285, Original class 285
0.670454545455
Attacking alexnet
Prediction before adding the scratch: 944, Original class 944
0.670454545455
Attacking alexnet
Prediction before adding the scratch: 944, Original class 944
0.670454545455
Attacking alexnet
Prediction before adding the scratch: 243, Origina

Prediction before adding the scratch: 321, Original class 321
0.674157303371
Attacking alexnet
Prediction before adding the scratch: 595, Original class 595
0.674157303371
Attacking alexnet
Prediction before adding the scratch: 843, Original class 843
0.674157303371
Attacking alexnet
Prediction before adding the scratch: 876, Original class 834
0.674157303371
Attacking alexnet
Prediction before adding the scratch: 991, Original class 991
0.666666666667
Attacking alexnet
Prediction before adding the scratch: 693, Original class 646
0.666666666667
Attacking alexnet
Prediction before adding the scratch: 69, Original class 69
0.666666666667
Attacking alexnet
Prediction before adding the scratch: 258, Original class 258
0.666666666667
Attacking alexnet
Prediction before adding the scratch: 292, Original class 292
0.666666666667
Attacking alexnet
Prediction before adding the scratch: 7, Original class 7
0.666666666667
Attacking alexnet
Prediction before adding the scratch: 354, Original clas

Prediction before adding the scratch: 129, Original class 130
0.652631578947
Attacking alexnet
Prediction before adding the scratch: 913, Original class 821
0.652631578947
Attacking alexnet
Prediction before adding the scratch: 314, Original class 314
0.645833333333
Attacking alexnet
Prediction before adding the scratch: 800, Original class 800
0.645833333333
Attacking alexnet
Prediction before adding the scratch: 681, Original class 881
0.645833333333
Attacking alexnet
Prediction before adding the scratch: 659, Original class 809
0.645833333333
Attacking alexnet
Prediction before adding the scratch: 367, Original class 367
0.645833333333
Attacking alexnet
Prediction before adding the scratch: 302, Original class 78
0.645833333333
Attacking alexnet
Prediction before adding the scratch: 306, Original class 606
0.645833333333
Attacking alexnet
Prediction before adding the scratch: 336, Original class 336
0.645833333333
Attacking alexnet
Prediction before adding the scratch: 701, Original

In [12]:
type(len(correct_preds) / 1.0 * (len(incorrect_preds) + len(correct_preds)))

float

# FOOLING RATES BELOW

## Find the images in bunches of 10 in the logs. Find their corresponding pickles. Find the ratio of them.

# RESNET FIRST

In [77]:
import os

# ResNet-50
pickled = '/pickled_resnet50/'
filenames_path = '/log_resnet50'
log_filenames = os.listdir(os.getcwd() + filenames)
pickled_filenames = os.listdir(os.getcwd() + pickled)
images_count = {}

for log in log_filenames:
    images_count[log.split('imageName_')[1].split('_fitness')[0]] = 0

for log in log_filenames:
    imagename = log.split('imageName_')[1].split('_fitness')[0]
    images_count[imagename] += 1
    
i = 0
successful_pickled = 0
for key, val in zip(images_count.keys(), images_count.values()):
    if (val == 10 or i == 100):
        i += 1
        fname = [s for s in pickled_filenames if (key in s)]
        print len(fname)
        successful_pickled += len(fname)
        finder = 0
        
print successful_pickled #627/1000 for ResNet

10
4
3
3
5
2
9
9
3
2
2
10
10
7
3
0
9
9
7
10
6
6
5
0
5
6
3
9
6
6
10
9
10
5
10
5
2
7
1
4
2
9
9
3
5
1
7
1
10
6
0
6
7
6
2
4
6
6
7
9
10
7
7
1
8
8
5
8
9
7
10
0
10
6
8
3
10
5
6
9
2
6
8
5
4
1
7
5
5
4
0
8
7
6
10
5
7
1
9
2
4
4
8
4
7
6
9
8
627


# VGG-19 NOW

In [126]:
import os

# ResNet-50
pickled = '/pickled_vgg19/'
filenames_path = '/log_vgg19'
log_filenames = os.listdir(os.getcwd() + filenames_path)
pickled_filenames = os.listdir(os.getcwd() + pickled)
images_count = {}

for log in log_filenames:
    if ('.JPEG' in log):
        images_count[log.split('imageName_')[1].split('_fitness')[0]] = 0
    
for log in log_filenames:
    if ('.JPEG' in log):
        imagename = log.split('imageName_')[1].split('_fitness')[0]
        images_count[imagename] += 1
    
i = 0
successful_pickled = 0
for key, val in zip(images_count.keys(), images_count.values()):
    print key, val
    if (val == 10):
        i += 1
        fname = [s for s in pickled_filenames if (key in s)]
#         print len(fname)
        successful_pickled += len(fname)
        finder = 0
        
print successful_pickled, i

ILSVRC2012_val_00001875.JPEG 10
ILSVRC2012_val_00022060.JPEG 10
ILSVRC2012_val_00015132.JPEG 1
ILSVRC2012_val_00009746.JPEG 10
ILSVRC2012_val_00032372.JPEG 10
ILSVRC2012_val_00000564.JPEG 10
ILSVRC2012_val_00025666.JPEG 10
ILSVRC2012_val_00034126.JPEG 10
ILSVRC2012_val_00016178.JPEG 10
ILSVRC2012_val_00009146.JPEG 10
ILSVRC2012_val_00026290.JPEG 10
ILSVRC2012_val_00033601.JPEG 10
ILSVRC2012_val_00016472.JPEG 1
ILSVRC2012_val_00043039.JPEG 10
ILSVRC2012_val_00001470.JPEG 10
ILSVRC2012_val_00030876.JPEG 10
ILSVRC2012_val_00014313.JPEG 10
ILSVRC2012_val_00023541.JPEG 10
ILSVRC2012_val_00020692.JPEG 10
ILSVRC2012_val_00001884.JPEG 10
ILSVRC2012_val_00016554.JPEG 10
ILSVRC2012_val_00017630.JPEG 10
ILSVRC2012_val_00033252.JPEG 10
ILSVRC2012_val_00022033.JPEG 10
ILSVRC2012_val_00042011.JPEG 10
ILSVRC2012_val_00047772.JPEG 10
ILSVRC2012_val_00018284.JPEG 2
ILSVRC2012_val_00049706.JPEG 10
ILSVRC2012_val_00038351.JPEG 10
ILSVRC2012_val_00042482.JPEG 9
ILSVRC2012_val_00026186.JPEG 10
ILSVRC2012_v

# FINALLY INCEPTION-V3

## The image file name isnt in the pickle files. We have to find all images with a fixed original label, and then search for files matching that in the pickles.

In [123]:
import os
import re

# INCEPTION-V3
pickled = '/pickled_image_parts_abs/'
filenames_path = '/log_abs'
log_filenames = os.listdir(os.getcwd() + filenames_path)
pickled_filenames = os.listdir(os.getcwd() + pickled)
images_count = {}

for log in log_filenames:
    if ('.JPEG' in log and 'original' in log):
        numbers = re.findall(r'\d+', log)
        target = numbers[3]
        original = numbers[4]
        y1 = numbers[-1]
        x1 = numbers[-3]
        y0 = numbers[-5]
        x0 = numbers[-7]
        strname = '_original=' + str(original)
        images_count[strname] = 0
    
for log in log_filenames:
    if ('.JPEG' in log and 'original' in log):
        numbers = re.findall(r'\d+', log)
        target = numbers[3]
        original = numbers[4]
        y1 = numbers[-1]
        x1 = numbers[-3]
        y0 = numbers[-5]
        x0 = numbers[-7]
        strname = '_original=' + str(original)
        images_count[strname] += 1
        
i = 0
successful_pickled = 0
for key, val in zip(images_count.keys(), images_count.values()):
#     print key, val
    if (val == 10):
        i += 1
        fname = [s for s in pickled_filenames if (key in s)]
        print len(fname)
        successful_pickled += len(fname)
#         finder = 0

3
9
9
1
9
0
8
10
4
6
7
3
7
10
19
8
7
8
1
7
2
5
10
10
0
27
5
9
1
5
8
9
7
7
5
6
8
2
10
14
14
7
10
7
10
9
8
2
3
4
2
2
6
5
1
8
3
8
8
8
1
1
2
17
15
5
7
1
3
9
5
2
10
3
3
3
8
10
8
3
10
6
3
10
8
8
10
1
0
6
24
16
9
10
8
9
9
6
68
10


In [128]:
images_count


{'ILSVRC2012_val_00000564.JPEG': 10,
 'ILSVRC2012_val_00001470.JPEG': 10,
 'ILSVRC2012_val_00001875.JPEG': 10,
 'ILSVRC2012_val_00001884.JPEG': 10,
 'ILSVRC2012_val_00003239.JPEG': 10,
 'ILSVRC2012_val_00004103.JPEG': 10,
 'ILSVRC2012_val_00004294.JPEG': 10,
 'ILSVRC2012_val_00004395.JPEG': 10,
 'ILSVRC2012_val_00004963.JPEG': 10,
 'ILSVRC2012_val_00005492.JPEG': 10,
 'ILSVRC2012_val_00006887.JPEG': 10,
 'ILSVRC2012_val_00007016.JPEG': 10,
 'ILSVRC2012_val_00007222.JPEG': 10,
 'ILSVRC2012_val_00007679.JPEG': 10,
 'ILSVRC2012_val_00008115.JPEG': 10,
 'ILSVRC2012_val_00008266.JPEG': 10,
 'ILSVRC2012_val_00008361.JPEG': 10,
 'ILSVRC2012_val_00008813.JPEG': 1,
 'ILSVRC2012_val_00009146.JPEG': 10,
 'ILSVRC2012_val_00009746.JPEG': 10,
 'ILSVRC2012_val_00009886.JPEG': 10,
 'ILSVRC2012_val_00010348.JPEG': 9,
 'ILSVRC2012_val_00011711.JPEG': 8,
 'ILSVRC2012_val_00012812.JPEG': 10,
 'ILSVRC2012_val_00012948.JPEG': 10,
 'ILSVRC2012_val_00013293.JPEG': 2,
 'ILSVRC2012_val_00013386.JPEG': 10,
 'ILS